In [1]:
#start spark outside of its home directory

import findspark

In [2]:
#place your path to Spark: findspark.init('PATH-TO-SPARK-FOLDER')

findspark.init('/home/user/spark-3.3.0-bin-hadoop3')

In [3]:
import pyspark

In [6]:
#create spark session and give name the app.

from pyspark.sql import SparkSession

In [7]:
spark = SparkSession.builder.appName('NLP').getOrCreate()

22/11/01 17:05:50 WARN Utils: Your hostname, linux-laptop resolves to a loopback address: 127.0.1.1; using 192.168.1.146 instead (on interface wlp54s0)
22/11/01 17:05:50 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/11/01 17:05:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/11/01 17:05:51 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [8]:
#import the data 

data = spark.read.csv("SMSSpamCollection",inferSchema=True,sep='\t')

In [9]:
#data column titles in CSV file is meaningless; give names: class and text respectively

data = data.withColumnRenamed('_c0','class').withColumnRenamed('_c1','text')

In [10]:
data.show()

+-----+--------------------+
|class|                text|
+-----+--------------------+
|  ham|Go until jurong p...|
|  ham|Ok lar... Joking ...|
| spam|Free entry in 2 a...|
|  ham|U dun say so earl...|
|  ham|Nah I don't think...|
| spam|FreeMsg Hey there...|
|  ham|Even my brother i...|
|  ham|As per your reque...|
| spam|WINNER!! As a val...|
| spam|Had your mobile 1...|
|  ham|I'm gonna be home...|
| spam|SIX chances to wi...|
| spam|URGENT! You have ...|
|  ham|I've been searchi...|
|  ham|I HAVE A DATE ON ...|
| spam|XXXMobileMovieClu...|
|  ham|Oh k...i'm watchi...|
|  ham|Eh u remember how...|
|  ham|Fine if thats th...|
| spam|England v Macedon...|
+-----+--------------------+
only showing top 20 rows



In [11]:
#Clean and Prepare the Data

#make use of the length function found in Spark to get size of ham/spam messages

from pyspark.sql.functions import length

data = data.withColumn('length',length(data['text']))

In [12]:
data.show()

+-----+--------------------+------+
|class|                text|length|
+-----+--------------------+------+
|  ham|Go until jurong p...|   111|
|  ham|Ok lar... Joking ...|    29|
| spam|Free entry in 2 a...|   155|
|  ham|U dun say so earl...|    49|
|  ham|Nah I don't think...|    61|
| spam|FreeMsg Hey there...|   147|
|  ham|Even my brother i...|    77|
|  ham|As per your reque...|   160|
| spam|WINNER!! As a val...|   157|
| spam|Had your mobile 1...|   154|
|  ham|I'm gonna be home...|   109|
| spam|SIX chances to wi...|   136|
| spam|URGENT! You have ...|   155|
|  ham|I've been searchi...|   196|
|  ham|I HAVE A DATE ON ...|    35|
| spam|XXXMobileMovieClu...|   149|
|  ham|Oh k...i'm watchi...|    26|
|  ham|Eh u remember how...|    81|
|  ham|Fine if thats th...|    56|
| spam|England v Macedon...|   155|
+-----+--------------------+------+
only showing top 20 rows



In [13]:
#perform some statistics; e.g. mean; see what the average length of a ham vs spam message is

data.groupby('class').mean().show()

+-----+-----------------+
|class|      avg(length)|
+-----+-----------------+
|  ham|71.45431945307645|
| spam|138.6706827309237|
+-----+-----------------+



In [14]:
#ML/NLP section inc feature engineering

#feature tools

from pyspark.ml.feature import Tokenizer,StopWordsRemover, CountVectorizer,IDF,StringIndexer

tokenizer = Tokenizer(inputCol="text", outputCol="token_text")
stopremove = StopWordsRemover(inputCol='token_text',outputCol='stop_tokens')
count_vec = CountVectorizer(inputCol='stop_tokens',outputCol='c_vec')
idf = IDF(inputCol="c_vec", outputCol="tf_idf")
ham_spam_to_num = StringIndexer(inputCol='class',outputCol='label')

from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vector

clean_up = VectorAssembler(inputCols=['tf_idf','length'],outputCol='features')

In [15]:
#classification model creation; Naive Bayes; others to look at: log. regress, random forests

from pyspark.ml.classification import NaiveBayes

nb = NaiveBayes()

#create pipeline

from pyspark.ml import Pipeline

data_prep_pipe = Pipeline(stages=[ham_spam_to_num,tokenizer,stopremove,count_vec,idf,clean_up])

In [16]:
#clean data

cleaner = data_prep_pipe.fit(data)

clean_data = cleaner.transform(data)

In [17]:
#start model evaluation

clean_data = clean_data.select(['label','features'])

clean_data.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(13424,[7,11,31,6...|
|  0.0|(13424,[0,24,297,...|
|  1.0|(13424,[2,13,19,3...|
|  0.0|(13424,[0,70,80,1...|
|  0.0|(13424,[36,134,31...|
|  1.0|(13424,[10,60,139...|
|  0.0|(13424,[10,53,103...|
|  0.0|(13424,[125,184,4...|
|  1.0|(13424,[1,47,118,...|
|  1.0|(13424,[0,1,13,27...|
|  0.0|(13424,[18,43,120...|
|  1.0|(13424,[8,17,37,8...|
|  1.0|(13424,[13,30,47,...|
|  0.0|(13424,[39,96,217...|
|  0.0|(13424,[552,1697,...|
|  1.0|(13424,[30,109,11...|
|  0.0|(13424,[82,214,47...|
|  0.0|(13424,[0,2,49,13...|
|  0.0|(13424,[0,74,105,...|
|  1.0|(13424,[4,30,33,5...|
+-----+--------------------+
only showing top 20 rows



In [18]:
#split data into training and test sets

(training,testing) = clean_data.randomSplit([0.7,0.3])

spam_predictor = nb.fit(training)

22/11/01 17:09:45 WARN DAGScheduler: Broadcasting large task binary with size 1144.3 KiB
22/11/01 17:09:45 WARN DAGScheduler: Broadcasting large task binary with size 1134.0 KiB


In [19]:
#create some test results

test_results = spam_predictor.transform(testing)

test_results.show()

22/11/01 17:10:03 WARN DAGScheduler: Broadcasting large task binary with size 1366.6 KiB
+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(13424,[0,1,3,9,1...|[-577.00395777797...|[1.0,4.3276712838...|       0.0|
|  0.0|(13424,[0,1,4,50,...|[-843.83845668000...|[1.0,3.1117370305...|       0.0|
|  0.0|(13424,[0,1,7,8,1...|[-886.79859591763...|[1.0,1.0481635956...|       0.0|
|  0.0|(13424,[0,1,7,8,1...|[-1186.3404580527...|[1.0,1.6890386781...|       0.0|
|  0.0|(13424,[0,1,9,14,...|[-536.74347994386...|[1.0,2.6219772119...|       0.0|
|  0.0|(13424,[0,1,12,33...|[-445.40865922441...|[1.0,3.1286639549...|       0.0|
|  0.0|(13424,[0,1,17,19...|[-805.62714222344...|[1.0,8.7842856299...|       0.0|
|  0.0|(13424,[0,1,18,20...|[-837.57234987852...|[1.0,8.4843351921...|       0.0|
|  0.0|(1

In [21]:
#compare label vs. prediction; looking at accuracy (acc)

from pyspark.ml.evaluation import MulticlassClassificationEvaluator

acc_eval = MulticlassClassificationEvaluator()
acc = acc_eval.evaluate(test_results)

print(f"Accuracy of Model at Predicting Spam Messages was {acc}")

22/11/01 17:14:33 WARN DAGScheduler: Broadcasting large task binary with size 1371.1 KiB
Accuracy of Model at Predicting Spam Messages was 0.9240702382171649
